In [2]:
import pandas
import numpy as np
import math

In [3]:
# Read CSV and create dataframe.
df = pandas.read_csv('sleepdata.csv',';')
df.describe()

,Heart rate (bpm),Steps,Air Pressure (Pa),Movements per hour,Time in bed (seconds),Time asleep (seconds),Time before sleep (seconds),Snore time,Weather temperature (°F)
count,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000,1037.000000
mean,2.279653,3442.538091,100.701929,51.439248,27761.055159,24924.869142,1268.614079,172.579653,47.281485
std,11.270408,3500.268331,5.523772,26.061872,5183.722344,4542.398533,1032.682986,442.679456,12.327545
min,0.000000,0.000000,0.000000,0.000000,0.000000,994.200000,9.500000,0.000000,26.800000
25%,0.000000,0.000000,100.600000,37.700000,26206.100000,23250.700000,542.200000,0.000000,32.000000
50%,0.000000,3219.000000,101.000000,49.300000,28740.100000,25562.200000,881.000000,0.000000,50.200000
75%,0.000000,5451.000000,101.400000,61.300000,30639.000000,27515.100000,1838.200000,130.000000,56.800000
max,67.000000,24901.000000,102.800000,414.000000,43609.200000,37503.900000,5827.200000,5885.000000,82.200000


In [4]:
df.head(-10)

,Start,End,Sleep Quality,Regularity,Mood,Heart rate (bpm),Steps,Alarm mode,Air Pressure (Pa),City,...,Time in bed (seconds),Time asleep (seconds),Time before sleep (seconds),Window start,Window stop,Did snore,Snore time,Weather temperature (°F),Weather type,Notes
0,2016-02-08 23:41:35,2016-02-09 06:14:10,67%,0%,NaN,0,2349,Normal,101.9,NaN,...,23555.1,21513.6,471.1,2016-02-09 06:00:00,2016-02-09 06:30:00,False,0.0,32.0,No weather,NaN
1,2016-02-10 00:18:58,2016-02-10 06:11:31,68%,0%,NaN,0,4549,Normal,102.0,NaN,...,21152.9,19531.2,423.1,2016-02-10 06:00:00,2016-02-10 06:30:00,False,0.0,32.0,No weather,NaN
2,2016-02-11 23:47:33,2016-02-12 06:03:08,54%,94%,NaN,0,2558,Normal,102.0,NaN,...,22535.7,20357.2,450.7,2016-02-12 06:00:00,2016-02-12 06:30:00,False,0.0,32.0,No weather,NaN
3,2016-02-13 00:03:54,2016-02-13 06:43:22,61%,93%,NaN,0,6663,Normal,102.0,NaN,...,23967.7,21411.2,479.4,2016-02-13 06:15:00,2016-02-13 06:45:00,False,0.0,32.0,No weather,NaN
4,2016-02-15 23:52:50,2016-02-16 06:27:52,75%,96%,NaN,0,6423,Normal,101.4,NaN,...,23701.3,22042.2,474.0,2016-02-16 06:05:00,2016-02-16 06:35:00,False,0.0,32.0,No weather,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,2021-02-08 23:48:09,2021-02-09 08:15:31,95%,92%,OK,60,4663,Normal,101.4,Pismo Beach,...,30441.6,24860.6,608.8,2021-02-09 07:30:00,2021-02-09 08:00:00,True,300.4,50.4,Cloudy,Aderrall:Allergy medicine before bed:Congested...
1023,2021-02-09 23:01:27,2021-02-10 07:24:21,84%,89%,OK,63,6479,Normal,101.5,Pismo Beach,...,30173.5,27457.9,603.5,2021-02-10 07:15:00,2021-02-10 07:45:00,True,154.4,47.3,Sunny,Allergy medicine before bed:Congested sinuses ...
1024,2021-02-12 00:11:10,2021-02-12 08:56:27,96%,82%,Good,52,6128,No alarm,100.0,Los Angeles,...,31516.7,30886.4,630.3,NaN,NaN,True,600.0,50.7,Fog,Allergy medicine before bed:Congested sinuses ...
1025,2021-02-14 00:56:02,2021-02-14 08:30:41,69%,90%,OK,66,4819,No alarm,101.1,San Diego,...,27279.7,22096.5,1273.1,NaN,NaN,True,1823.0,55.6,Sunny,Allergy medicine before bed:Congested sinuses ...


In [5]:
# Clean data
df['Sleep Quality'] = df['Sleep Quality'].apply(lambda x: int(x.replace("%","")))
df['Regularity'] = df['Regularity'].apply(lambda x: int(x.replace("%","")))
df = df[df['Sleep Quality'] > 0 ]
df = df[df['Regularity'] > 0]
df = df[df['Steps'] > 0 ]
df = df[df['Time in bed (seconds)'] > 0]

In [6]:
# Function to Change seconds to hours and minutes.
def hours_mins(x):
    mins = round(x/60)
    hours = round(((x/60)/60), 2)
    hrs_mins = (math.floor(hours), round(mins % 60))
    #print(f'Total minutes: {round(mins,1)}')
    #print(f'Total hours: {round(hours,1)}')
    #print(hrs_mins)
    return (mins,hours,hrs_mins)


In [7]:
# Apply hours_mins function to series to convert from seconds to minutes
df['Time_in_bed_(minutes)'] = df['Time in bed (seconds)'].apply(lambda x: hours_mins(x)[0])
df['Time_asleep_(minutes)'] = df['Time asleep (seconds)'].apply(lambda x: hours_mins(x)[0])
df['Time_before_sleep_(minutes)'] = df['Time before sleep (seconds)'].apply(lambda x: hours_mins(x)[0])
df['Snore_time_(minutes)'] = df['Snore time'].apply(lambda x: hours_mins(x)[0])

# Delete old time columns
df.drop(columns=['Time asleep (seconds)', 'Time in bed (seconds)', 'Time before sleep (seconds)', 'Snore time'])



,Start,End,Sleep Quality,Regularity,Mood,Heart rate (bpm),Steps,Alarm mode,Air Pressure (Pa),City,...,Window start,Window stop,Did snore,Weather temperature (°F),Weather type,Notes,Time_in_bed_(minutes),Time_asleep_(minutes),Time_before_sleep_(minutes),Snore_time_(minutes)
2,2016-02-11 23:47:33,2016-02-12 06:03:08,54,94,NaN,0,2558,Normal,102.0,NaN,...,2016-02-12 06:00:00,2016-02-12 06:30:00,False,32.0,No weather,NaN,376,339,8,0
3,2016-02-13 00:03:54,2016-02-13 06:43:22,61,93,NaN,0,6663,Normal,102.0,NaN,...,2016-02-13 06:15:00,2016-02-13 06:45:00,False,32.0,No weather,NaN,399,357,8,0
4,2016-02-15 23:52:50,2016-02-16 06:27:52,75,96,NaN,0,6423,Normal,101.4,NaN,...,2016-02-16 06:05:00,2016-02-16 06:35:00,False,32.0,No weather,NaN,395,367,8,0
5,2016-02-16 23:59:44,2016-02-17 06:06:09,72,96,NaN,0,3190,Normal,100.3,NaN,...,2016-02-17 05:45:00,2016-02-17 06:15:00,False,32.0,No weather,NaN,366,359,7,0
6,2016-02-18 23:37:39,2016-02-19 06:23:58,79,95,NaN,0,2893,Normal,101.5,NaN,...,2016-02-19 06:10:00,2016-02-19 06:40:00,False,32.0,No weather,NaN,406,398,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,2021-02-22 23:09:43,2021-02-23 07:59:15,97,85,OK,0,3680,Normal,101.7,Pismo Beach,...,2021-02-23 07:15:00,2021-02-23 07:45:00,True,53.6,Sunny,Congested sinuses :Up on iPhone/tablet before bed,530,489,11,5
1033,2021-02-23 23:34:49,2021-02-24 08:07:43,91,89,OK,0,7117,Normal,101.5,Pismo Beach,...,2021-02-24 08:00:00,2021-02-24 08:30:00,True,53.8,Sunny,Allergy medicine before bed:Congested sinuses ...,513,479,15,13
1034,2021-02-24 23:15:55,2021-02-25 07:35:36,89,94,Good,53,4635,Normal,101.7,Pismo Beach,...,2021-02-25 07:30:00,2021-02-25 08:00:00,True,53.4,Sunny,Allergy medicine before bed:Congested sinuses ...,500,470,10,12
1035,2021-02-25 23:57:18,2021-02-26 07:54:59,90,93,OK,50,6302,Normal,101.8,Pismo Beach,...,2021-02-26 07:45:00,2021-02-26 08:15:00,True,50.4,Sunny,Aderrall:Bath:Long day at work. No free time.:...,478,414,46,1


In [8]:
print(df.head(-20))

                    Start                  End  Sleep Quality  Regularity  \
2     2016-02-11 23:47:33  2016-02-12 06:03:08             54          94   
3     2016-02-13 00:03:54  2016-02-13 06:43:22             61          93   
4     2016-02-15 23:52:50  2016-02-16 06:27:52             75          96   
5     2016-02-16 23:59:44  2016-02-17 06:06:09             72          96   
6     2016-02-18 23:37:39  2016-02-19 06:23:58             79          95   
...                   ...                  ...            ...         ...   
1012  2021-01-26 23:02:48  2021-01-27 07:42:55             91          96   
1013  2021-01-27 23:03:47  2021-01-28 07:53:03             95          97   
1014  2021-01-28 23:12:49  2021-01-29 07:54:41             91          98   
1015  2021-01-30 01:06:43  2021-01-30 08:39:29             70          92   
1016  2021-01-31 23:10:51  2021-02-01 08:13:15             91          79   

      Mood  Heart rate (bpm)  Steps Alarm mode  Air Pressure (Pa)  \
2     

In [9]:
# Check correlations
"""
df_notes = df[df['Notes'].notna()]
df_smoked = df_notes[df_notes['Notes'].str.contains('Smoked')]
df_no_smoke = df_notes[~df_notes['Notes'].str.contains('Smoked')]
df_smoked.describe()
"""


"\ndf_notes = df[df['Notes'].notna()]\ndf_smoked = df_notes[df_notes['Notes'].str.contains('Smoked')]\ndf_no_smoke = df_notes[~df_notes['Notes'].str.contains('Smoked')]\ndf_smoked.describe()\n"

In [10]:
"""
quality_delta = df_smoked['Sleep Quality'].mean() - df_no_smoke['Sleep Quality'].mean()
quality_delta = round(quality_delta,2)
time_to_sleep = df_smoked['Time before sleep (seconds)'].mean() - df_no_smoke['Time before sleep (seconds)'].mean()
time_to_sleep = round(time_to_sleep,2)
print(f"Sleep quality change on nights after drinking coffee: {quality_delta}%")
print(f"Chane in time it takes to fall asleep after drinking coffee: {time_to_sleep}min")
"""

'\nquality_delta = df_smoked[\'Sleep Quality\'].mean() - df_no_smoke[\'Sleep Quality\'].mean()\nquality_delta = round(quality_delta,2)\ntime_to_sleep = df_smoked[\'Time before sleep (seconds)\'].mean() - df_no_smoke[\'Time before sleep (seconds)\'].mean()\ntime_to_sleep = round(time_to_sleep,2)\nprint(f"Sleep quality change on nights after drinking coffee: {quality_delta}%")\nprint(f"Chane in time it takes to fall asleep after drinking coffee: {time_to_sleep}min")\n'

In [11]:
# Function to show sleep quality with/without and delta
# df is the cleaned dataframe
# notesTerm is the Notes term to filter
def sleepQualityDelta(df,notesTerm):
    # Check correlations
    df_notes = df[df['Mood'].notna()]
    df_temp_with = df_notes[df_notes['Notes'].notna()]
    df_with = df_temp_with[df_temp_with['Notes'].str.contains(notesTerm)]
    #df_with = df_notes.loc[df_notes['Notes'].notna(), df_notes['Notes'].str.contains(notesTerm)]
    df_notes['Notes'] = df_notes['Notes'].replace(np.nan, 'NA')
    #df_notes['Notes'].fillna('NA')
    #print(df_notes.head(10))
    df_without = df_notes[~df_notes['Notes'].str.contains(notesTerm)]
    df_with.describe()

    # Mood
    total_with = len(df_with.index)
    ok_mood = len(df_with[df_with['Mood'] == 'OK'].index)
    ok_mood_percent = (ok_mood/total_with)*100
    good_mood = len(df_with[df_with['Mood'] == 'Good'].index)
    good_mood_percent = (good_mood/total_with)*100
    bad_mood = len(df_with[df_with['Mood'] == 'Bad'].index)
    bad_mood_percent = (bad_mood/total_with)*100

    total_without = len(df_without.index)
    ok_mood_wo = len(df_without[df_without['Mood'] == 'OK'].index)
    ok_mood_percent_wo = (ok_mood_wo/total_without)*100
    good_mood_wo = len(df_without[df_without['Mood'] == 'Good'].index)
    good_mood_percent_wo = (good_mood_wo/total_without)*100
    bad_mood_wo = len(df_without[df_without['Mood'] == 'Bad'].index)
    bad_mood_percent_wo = (bad_mood_wo/total_without)*100

    ok_delta = ok_mood_percent - ok_mood_percent_wo
    ok_delta = round(ok_delta,2)
    good_delta = good_mood_percent - good_mood_percent_wo
    good_delta = round(good_delta,2)
    bad_delta = bad_mood_percent - bad_mood_percent_wo
    bad_delta = round(bad_delta,2)
    
    # Get percent of time with/without
    total_len = len(df_notes.index)
    len_with = len(df_with.index)
    percent_with = (len_with/total_len)*100
    percent_with = round(percent_with,2)

    # Change in sleep quality with/without
    quality_delta = df_with['Sleep Quality'].mean() - df_without['Sleep Quality'].mean()
    quality_delta = round(quality_delta,2)

    # Change in time it takes to fall asleep with/without
    time_to_sleep = df_with['Time_before_sleep_(minutes)'].mean() - df_without['Time_before_sleep_(minutes)'].mean()
    time_to_sleep = round(time_to_sleep,2)

    # Change in time asleep with/without
    time_asleep = df_with['Time_asleep_(minutes)'].mean() - df_without['Time_asleep_(minutes)'].mean()
    time_asleep = round(time_asleep,2)

    # Change in snoring time with/without
    time_snore = df_with['Snore_time_(minutes)'].mean() - df_without['Snore_time_(minutes)'].mean()
    time_snore = round(time_snore,2)

    print(f"Percent of days with {notesTerm}: {percent_with}%")
    print(f"Sleep quality change with {notesTerm}: {quality_delta}%")
    print(f"Change in time it takes to fall asleep on days with {notesTerm}: {time_to_sleep}min")
    print(f"Change in time asleep on days with {notesTerm}: {time_asleep}min")
    print(f"Change in snoring time on days with {notesTerm}: {time_snore}min")
    print(f"Change in neutral mood with {notesTerm}: {ok_delta}%")
    print(f"Change in good mood with {notesTerm}: {good_delta}%")
    print(f"Change in bad mood with {notesTerm}: {bad_delta}%")





In [23]:
sleepQualityDelta(df,'Allergy medicine')

Percent of days with Allergy medicine: 22.05%
Sleep quality change with Allergy medicine: 2.13%
Change in time it takes to fall asleep on days with Allergy medicine: -1.47min
Change in time asleep on days with Allergy medicine: 27.63min
Change in snoring time on days with Allergy medicine: 2.84min
Change in neutral mood with Allergy medicine: -10.4%
Change in good mood with Allergy medicine: 22.75%
Change in bad mood with Allergy medicine: -12.35%
ipykernel_launcher:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
df_notes = df[df['Notes'].notna()]
out = set()
for items in df_notes['Notes']:
    note_list = items.split(':') 
    #print(note)
    out.update({note for note in note_list})
print(out)


{'Headache', 'Long day at work. No free time.', 'Tired', 'Allergy medicine before bed', 'Aderrall', '2 or more coffee cups of caffeine ', 'Ate late', 'Meditated in the evening/night', 'Congested sinuses ', 'Sore lumbar', 'Drank more than 1 alc beverage ', 'Up on iPhone/tablet before bed', 'Sick', 'Worked out at gym', 'Sore neck', 'Nap', 'Bath', 'Not my bed', 'Stressful day', 'Played soccer', 'Very chill day', 'Up on computer before sleep', 'Pickle', 'Smoked weed'}
